# Smart Beta and Factor Investing

This assignment refers to the HBS case: **Smart Beta Exchange-Traded-Funds and Factor Investing**.

* The case is a good introduction to important pricing factors.
* It also gives useful introduction and context to ETFs, passive vs active investing, and so-called “smart beta” funds.

# 1. READING

1. Describe how each of the factors (other than MKT) is measured.1That is, each factor is a portfolio of stocks–which stocks are included in the factor portfolio?

1. Is the factor portfolio...
    * long-only
    * long-short
    * value-weighted
    * equally-weighted

1. What steps are taken in the factor construction to try to reduce the correlation between the factors?
5. What is the point of figures 1-6?
6. How is a “smart beta” ETF different from a traditional ETF?
7. Is it possible for all investors to have exposure to the “value” factor?
8. How does factor investing differ from traditional diversification?

#### Footnote:

If you need more info in how these factor portfolios are created, see Ken French’s website, and the follow- details: 

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/det_mom_factor.html

***

# 2. The Factors

### Data
Use the data found in `data/factor_pricing_data.xlsx`.

Factors: Monthly excess return data for the overall equity market, $\tilde{r}^{\text{MKT}}$.

* The column header to the market factor is `MKT` rather than `MKT-RF`, but it is indeed already in excess return form.

* The sheet also contains data on five additional factors.

* All factor data is already provided as excess returns

In [ ]:
import polars as pl
import numpy as np

# sheet names: 
# 'factors (excess returns)'
# 'portfolios (excess returns)'
# 'risk-free rate'

# factors
# Name
# MKT	Market
# SMB	Size
# HML	Value
# RMW	Profitability
# CMA	Investment
# UMD	Momentum
# RF	Risk-free rate

rets = pl.read_excel(
    '../data/factor_pricing_data_monthly.xlsx',
    sheet_name='factors (excess returns)',
)

rets.head()

Date,MKT,SMB,HML,RMW,CMA,UMD
date,f64,f64,f64,f64,f64,f64
1980-01-31,0.055,0.0188,0.0185,-0.0184,0.0189,0.0745
1980-02-29,-0.0123,-0.0162,0.0059,-0.0095,0.0292,0.0789
1980-03-31,-0.1289,-0.0697,-0.0096,0.0182,-0.0105,-0.0958
1980-04-30,0.0396,0.0105,0.0103,-0.0218,0.0034,-0.0048
1980-05-31,0.0526,0.02,0.0038,0.0043,-0.0063,-0.0118


### 1. 
Analyze the factors, similar to how you analyzed the three Fama-French factors in `Homework 4`.

You now have three additional factors, so let’s compare there univariate statistics. 

* mean
* volatility    
* Sharpe

In [2]:
def performance_metrics(returns, annualization=1, start_date=None, end_date=None): # , quantile=.05):
    # only calculate for columns containing numerical (factor) data, not "Date"
    # Filter the data by date if start_date or end_date are provided
    filtered_returns = returns
    import datetime

    if start_date is not None:
        if isinstance(start_date, str):
            start_date_dt = datetime.date.fromisoformat(start_date)
        else:
            start_date_dt = start_date
        filtered_returns = filtered_returns.filter(pl.col("Date") >= pl.lit(start_date_dt))

    if end_date is not None:
        if isinstance(end_date, str):
            end_date_dt = datetime.date.fromisoformat(end_date)
        else:
            end_date_dt = end_date
        filtered_returns = filtered_returns.filter(pl.col("Date") <= pl.lit(end_date_dt))

    factor_cols = [col for col in filtered_returns.columns if filtered_returns.schema[col] in [pl.Float64, pl.Float32, pl.Int64, pl.Int32]]
    means = filtered_returns.select([pl.col(factor_cols).mean() * annualization]).to_dict(as_series=False)
    vols = filtered_returns.select([pl.col(factor_cols).std() * np.sqrt(annualization)]).to_dict(as_series=False)
    sharpes = filtered_returns.select([
        (pl.col(col).mean() / pl.col(col).std() * np.sqrt(annualization)).alias(col) for col in factor_cols
    ]).to_dict(as_series=False)

    metrics = pl.DataFrame({
        "Factor": factor_cols,
        "Mean": [means[col][0] for col in factor_cols],
        "Vol": [vols[col][0] for col in factor_cols],
        "Sharpe": [sharpes[col][0] for col in factor_cols],
    })
    return metrics

metrics = performance_metrics(rets, annualization=12, start_date="2015-01-01")
metrics.head(10)

Factor,Mean,Vol,Sharpe
str,f64,f64,f64
"""MKT""",0.117872,0.157356,0.749078
"""SMB""",-0.023775,0.103166,-0.230455
"""HML""",-0.016303,0.129885,-0.12552
"""RMW""",0.0400125,0.072632,0.550896
"""CMA""",-0.009141,0.082072,-0.111373
"""UMD""",0.020119,0.137387,0.146438


### 2. 

Based on the factor statistics above, answer the following.
* Does each factor have a positive risk premium (positive expected excess return)? 
* How have the factors performed since the time of the case, (2015-present)?

For the 2015-present subsample, there are 3 factors with negative premia: size, value, and investment.

Profitability and momentum have fared better, especially the former with a Sharpe of 0.55.

### 3. 

Report the correlation matrix across the six factors.
* Does the construction method succeed in keeping correlations small?
* Fama and French say that HML is somewhat redundant in their 5-factor model. Does this seem to be the case?

In [3]:
# get the correlation matrix
corr_matrix = rets[["MKT", "SMB", "HML", "RMW", "CMA", "UMD"]].corr()
corr_matrix

MKT,SMB,HML,RMW,CMA,UMD
f64,f64,f64,f64,f64,f64
1.0,0.226997,-0.207918,-0.250639,-0.346542,-0.179352
0.226997,1.0,-0.021819,-0.411946,-0.051099,-0.06094
-0.207918,-0.021819,1.0,0.219401,0.676727,-0.215523
-0.250639,-0.411946,0.219401,1.0,0.138566,0.076694
-0.346542,-0.051099,0.676727,0.138566,1.0,0.000094
-0.179352,-0.06094,-0.215523,0.076694,0.000094,1.0


### 4. 

Report the tangency weights for a portfolio of these 6 factors.
* Which factors seem most important? And Least?
* Are the factors with low mean returns still useful?
* Re-do the tangency portfolio, but this time only include MKT, SMB, HML, and UMD. Which factors get high/low tangency weights now?

What do you conclude about the importance or unimportance of these styles?

***

# 3. Testing Modern LPMs

Consider the following factor models:
* CAPM: MKT
* Fama-French 3F: MKT, SMB, HML
* Fama-French 5F: MKT, SMB, HML, RMW, CMA
* AQR: MKT, HML, RMW, UMD

Our labeling of the last model as the **AQR** is just for concreteness. The firm is well-known for these factors and an unused case study discusses that further.

For instance, for the AQR model is...

$$
\mathbb{E}[\tilde{r}^i] 
= \beta^{i,\mathrm{MKT}} \, \mathbb{E}[\tilde{f}^{\mathrm{MKT}}] 
+ \beta^{i,\mathrm{HML}} \, \mathbb{E}[\tilde{f}^{\mathrm{HML}}] 
+ \beta^{i,\mathrm{RMW}} \, \mathbb{E}[\tilde{f}^{\mathrm{RMW}}] 
+ \beta^{i,\mathrm{UMD}} \, \mathbb{E}[\tilde{f}^{\mathrm{UMD}}]
$$

We will test these models with the time-series regressions. Namely, for each asset i, estimate the following regression to test the AQR model:

$$
\tilde{r}^i_t 
= \alpha^i 
+ \beta^{i,\mathrm{MKT}} \tilde{f}^{\mathrm{MKT}}_t 
+ \beta^{i,\mathrm{HML}} \tilde{f}^{\mathrm{HML}}_t 
+ \beta^{i,\mathrm{RMW}} \tilde{f}^{\mathrm{RMW}}_t 
+ \beta^{i,\mathrm{UMD}} \tilde{f}^{\mathrm{UMD}}_t 
+ \varepsilon_t
$$

### Data

* Monthly excess return data on `n=49` equity portfolios sorted by their industry. Denote these as $\tilde{r}^i$ , for $n = 1, . . . .$

* You do NOT need the risk-free rate data. It is provided only for completeness. The other two tabs are already in terms of excess returns.

### 1. 

Test the AQR 4-Factor Model using the time-series test. (We are not doing the cross-sectional regression tests.)

For each regression, report the estimated α and r-squared.

### 2. 

Calculate the mean-absolute-error of the estimated alphas.

$$\text{MAE} = \frac{1}{n}\sum_{i=1}^n|\tilde{\alpha}^i|$$

* If the pricing model worked, should these alpha estimates be large or small? Why?

* Based on your MAE stat, does this seem to support the pricing model or not?

### 2. 

Test the CAPM, FF 3-Factor Model and the the FF 5-Factor Model.
   * Report the MAE statistic for each of these models and compare it with the AQR Model MAE.
   * Which model fits best?
   
### 3. 

Does any particular factor seem especially important or unimportant for pricing? Do you think Fama and French should use the Momentum Factor?

### 4. 

This does not matter for pricing, but report the average (across $n$ estimations) of the time-series regression r-squared statistics.
   * Do this for each of the three models you tested.
   * Do these models lead to high time-series r-squared stats? That is, would these factors be good in a Linear Factor Decomposition of the assets?

### 5. 

We tested three models using the time-series tests (focusing on the time-series alphas.) Re-test these models, but this time use the cross-sectional test.

* Report the time-series premia of the factors (just their sample averages,) and compare to the cross-sectionally estimated premia of the factors. Do they differ substantially?4
* Report the MAE of the cross-sectional regression residuals for each of the four models. How do they compare to the MAE of the time-series alphas?

#### Footnote:

Recall that we found in `Homework 4` that the market premium went from being strongly positive to strongly negative when estimated in the cross-section.

***